In [ ]:
import joblib

from film_recoomendations.data_access import get_film_ratings
from film_recoomendations.ml_fitting import (
    create_transformed_features,
    evaluate_rating_predictions,
    fit_pipeline,
    make_predictions,
    split_train_test,
)
from film_recoomendations.preprocessing import clean_data, create_features

In [ ]:
film_ratings_df = get_film_ratings()
df = clean_data(film_ratings_df)
features_df, _missing_list = create_features(df)
features_df.to_csv("./ratings_with_tmdb_features.csv", index=False)
features_df, target_series = create_transformed_features(features_df)

In [ ]:
X_train, y_train, X_test, y_test = split_train_test(features_df)
pipe = fit_pipeline(X_train, y_train)
joblib.dump(pipe, "./letterboxd_tmdb_rating_model.joblib")
eval_df, pred_clipped = make_predictions(pipe, X_test, y_test)
results = evaluate_rating_predictions(y_test, pred_clipped, eval_df)